![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# STEP 1: Reading in the dataset as a pandas Data Frame
bank_marketing_df = pd.read_csv("bank_marketing.csv")

# STEP 2: Splitting the data
client = bank_marketing_df[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]

campaign = bank_marketing_df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome", "month", "day"]] # Added "month" and "day" columns

# FIX: Correct the column name typo from "euribor_thee_months" to "euribor_three_months"
economics = bank_marketing_df[["client_id", "cons_price_idx", "euribor_three_months"]]

# STEP 3: Cleaning columns

# Replacing "." with "_" in the "education" column
client["education"] = client["education"].str.replace(".", "_")

# FIX: Use .replace() instead of .str.replace() to replace "unknown" with np.NaN
client["education"] = client["education"].replace("unknown", np.NaN)

# Replacing "." with "_" in the "job" column
client["job"] = client["job"].str.replace(".", "_")

# Converting "credit_default" and "mortgage" to boolean
for col in ["credit_default", "mortgage"]:
    client[col] = client[col].map({"yes": 1, "no": 0, "unknown": 0})
    client[col] = client[col].astype(bool)

# Converting "campaign_outcome" to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no": 0})
campaign["campaign_outcome"] = campaign["campaign_outcome"].astype(bool)

# Converting "previous_outcome" to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, "failure": 0, "nonexistent": 0})
campaign["previous_outcome"] = campaign["previous_outcome"].astype(bool)

# STEP 4: Creating new columns

# Adding a new column called "year" where all values are set as "2022"
campaign["year"] = "2022"

# Converting the "day" column to a string
campaign["day"] = campaign["day"].astype(str)

# Adding columns together with "-" to separate the values
campaign["last_contact_date"] = campaign["year"] + "-" + campaign['month'] + "-" + campaign["day"]

# Converting "last_contact_date" using the format "%Y-%b-%d"
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format = "%Y-%b-%d")

# STEP 5: Deleting columns

# Removing "month", "day", and "year" columns
campaign.drop(columns = ["month", "day", "year"], inplace = True)

#STEP 6: Saving the data
client.to_csv("client.csv", index = False)
campaign.to_csv("campaign.csv", index = False)
economics.to_csv("economics.csv", index = False)

In [2]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64
